# **MergingShit™ V2 based on ColabADMIXTOOLS Version 4.8a**
Simplified, use it to merge shit. Done by pepsimanfire (Discord) but all credits to Florio who did the OG collab. florus14 his Discord. There's no point in merging on RStudio or whatsoever, merging requires TOO MUCH computational power.

# **[1] Install Software**

You can run with CPU if merging with the HO, but only with TPU 2.8 Runtime if 1240k. Pro tip: if you run out of TPU time, you can always use a different Google account, or create a new one.

In [ ]:
import os
from subprocess import run, PIPE

# @markdown **Select your preferences for the setup process**

# @markdown Do you want to install ADMIXTOOLS?
install_admixtools = True  # @param {type:"boolean"}

# Function to check if a command exists
def command_exists(command):
    return run(['which', command], stdout=PIPE, stderr=PIPE).returncode == 0

# Install R if not already installed
if not command_exists('R'):
    print("Installing R...")
    !sudo apt-get update -y
    !sudo apt-get install -y r-base
else:
    print("R is already installed.")

# Install rpy2 if not already installed
try:
    import rpy2
    print("rpy2 is already installed.")
except ImportError:
    print("Installing rpy2...")
    !pip install rpy2

# Load the rpy2 extension for Python
%load_ext rpy2.ipython

# Check for PLINK
if not os.path.exists('/content/plink'):
    print("Installing PLINK (latest stable version)...")
    !wget https://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20241022.zip -O plink.zip
    !unzip -o plink.zip -d /content/
    os.remove("plink.zip")
    print("PLINK installation complete!")
else:
    print("PLINK is already installed.")

# Check and install ADMIXTOOLS
if install_admixtools:
    if not os.path.exists('/content/AdmixTools'):
        print("Installing ADMIXTOOLS...")
        !apt-get update
        !apt-get install -y build-essential libgsl-dev libopenblas-dev gfortran liblapacke-dev \
                           libssl-dev libffi-dev libncurses5-dev zlib1g zlib1g-dev \
                           libreadline-dev libbz2-dev libsqlite3-dev
        !rm -rf AdmixTools
        !git clone https://github.com/DReichLab/AdmixTools.git
        %cd AdmixTools/src
        !sed -i "s/-Wimplicit/-Wimplicit -fcommon/" Makefile
        !make clobber
        !make all LDLIBS="-llapacke -llapack -lgsl -lopenblas -lm -lnick"
        !make install
        %cd /content/
        print("ADMIXTOOLS installation complete!")
    else:
        print("ADMIXTOOLS is already installed.")


print("Setup complete!")

# Add this right after ADMIXTOOLS installation
os.environ['PATH'] += os.pathsep + '/content/AdmixTools/bin'
print("ADMIXTOOLS binaries added to PATH")

# **[2] Your DNA File Upload**

In [ ]:
# @title Upload 23andMe Raw Data
from google.colab import files

uploaded = files.upload()

# Assuming the user will upload only one genome file, you can check if a file has been uploaded successfully.
if uploaded:
    filename = next(iter(uploaded))
    print(f"File '{filename}' uploaded successfully.")
else:
    print("No file uploaded.")


**23andMe formatted data works best for conversion. Please explore [DNA Kit
Studio](https://www.dnagenics.com/dna-kit-studio/) on how to make this conversion possible. Some users have also found success with [this](https://tendna.com/en/dna-converter) converter as well.**

# **[3] Dataset Selection and snplist download**

**The 1240K has more coverage than the 1240K+HO. However, the 1240K+HO has more modern samples. You can download one after the other if you would like to explore both datasets.**

In [ ]:
# Clone the repo into /content/snplist
!git clone https://github.com/astralprojectionfont/snplist.git

# Verify the download
!ls -la snplist/

# @title Download 1240K or HO Datasets (AADR v62)
# @markdown Select the dataset you want to download:

dataset_choice = "1240K" #@param ["1240K", "1240K+HO"]

# Define the URLs and corresponding filenames for each dataset
dataset_files = {
    "1240K": [
        ("https://dataverse.harvard.edu/api/access/datafile/10537413", "aadr_v62.0_1240K_public.anno"),
        ("https://dataverse.harvard.edu/api/access/datafile/10537126", "aadr_v62.0_1240K_public.geno"),
        ("https://dataverse.harvard.edu/api/access/datafile/10537414", "aadr_v62.0_1240K_public.ind"),
        ("https://dataverse.harvard.edu/api/access/datafile/10537415", "aadr_v62.0_1240K_public.snp"),
    ],
    "1240K+HO": [
        ("https://dataverse.harvard.edu/api/access/datafile/10537417", "aadr_v62.0_HO_public.anno"),
        ("https://dataverse.harvard.edu/api/access/datafile/10537419", "aadr_v62.0_HO_public.geno"),
        ("https://dataverse.harvard.edu/api/access/datafile/10537420", "aadr_v62.0_HO_public.ind"),
        ("https://dataverse.harvard.edu/api/access/datafile/10537421", "aadr_v62.0_HO_public.snp"),
    ]
}

selected_files = dataset_files[dataset_choice]

print(f"Downloading {dataset_choice} dataset files...")
for url, filename in selected_files:
    !wget --no-check-certificate {url} -O /content/{filename}

print("Download complete!")

# **[4] Merging Yourself with the AADR**

The code will ask you for what dataset you want to use, if you want to trim using the snp list or not, the sex the samples are (1=Male 2=Female 0=Unknown) and for a name for the samples in the ind file. For the rest, it auto detects txts as samples. The more stuff you add, the more time it will take of course. I think it detects up to 10 samples, which I guess you will never use, that'd be TOO much time.
Many times, when merging samples from different companies, or just randomly, the script will fail, likely due to an issue with the samples and plink. I advise you to check the plink log, and to either try to remove the problematic raw file, or just merge one by one. In other cases, the conversion of the raw to the 23andme format can cause problems, it's kind of random really.
It's not possible to merge everything.

In [ ]:
import os
import subprocess

# Step 1: Select Dataset
while True:
    dataset_choice = input("Select dataset (1 for HO, 2 for 1240K): ").strip()
    if dataset_choice in ['1', '2']:
        dataset_name = "HO" if dataset_choice == '1' else "1240K"
        break
    print("Invalid choice. Please enter 1 for HO or 2 for 1240K.")

# Merge Dataset Parameters
geno1_path = f"aadr_v62.0_{dataset_name}_public.geno"
snp1_path = f"aadr_v62.0_{dataset_name}_public.snp"
ind1_path = f"aadr_v62.0_{dataset_name}_public.ind"

# Specify Inputs
genome_filepaths = [f"/content/{f}" for f in os.listdir("/content/") if f.endswith(".txt")]

if not genome_filepaths:
    raise ValueError("No valid genome TXT files found in /content/. Check your input files.")

sex_codes = []
individual_ids = []

for filepath in genome_filepaths:
    while True:
        sex = input(f"Enter sex for {filepath} (1 for male, 2 for female, 0 for unknown): ")
        if sex in ['1', '2', '0']:
            sex_codes.append(sex)
            break
        print("Invalid input. Please enter 1, 2, or 0.")

    name = input(f"Enter name for {filepath} (to be displayed in .ind file): ")
    individual_ids.append(name)

# Output prefix
output_base = input("Enter output base prefix: ").strip()

if not output_base:
    raise ValueError("Output base name cannot be empty.")

# Ask user if they want to trim using SNP list
trim_with_snplist = input("Do you want to trim using a SNP list? (yes/no): ").strip().lower() == "yes"
snplist_path = f"/content/snplist/{dataset_name}snplist.snplist" if trim_with_snplist else None

# Add Swap Memory to Prevent Out-of-Memory Issues
swap_file_path = "/swapfile"
os.system(f"fallocate -l 8G {swap_file_path}")
os.system(f"chmod 600 {swap_file_path}")
os.system(f"mkswap {swap_file_path}")
os.system(f"swapon {swap_file_path}")
print("Swap memory created successfully.")

# Step 2: Convert Each Sample Separately and Trim if Required
plink_file_prefixes = []
for i, genome_filepath in enumerate(genome_filepaths):
    sample_output_base = f"{output_base}_{i+1}"
    plink_file_prefixes.append(sample_output_base)

    print(f"Processing {genome_filepath}...")
    subprocess.run([
        "/content/plink", "--23file", genome_filepath, "--list-23-indels", "--allow-no-sex",
        "--make-bed", "--out", sample_output_base
    ], check=True)

    if trim_with_snplist:
        # Trim Using SNP List
        trimmed_prefix = f"{sample_output_base}_trimmed"
        subprocess.run([
            "/content/plink", "--bfile", sample_output_base, "--extract", snplist_path,
            "--make-bed", "--out", trimmed_prefix
        ], check=True)
        plink_file_prefixes[i] = trimmed_prefix  # Update prefix after trimming

    #Define unimportant IDs so it doesn't give an error
    family_ids = ["FID"] * len(genome_filepaths)  # Replace "FID" with actual family ID logic if needed
    father_ids = ["0"] * len(genome_filepaths)   # Assuming unknown father (modify as needed)
    mother_ids = ["0"] * len(genome_filepaths)   # Assuming unknown mother (modify as needed)
    phenotype_values = ["-9"] * len(genome_filepaths)  # Default missing phenotype (-9 for PLINK)

    #Define num_threads so it doesn't give an error
    num_threads = 2

    # Update .fam File
    fam_file_path = f"/content/{plink_file_prefixes[i]}.fam"
    new_fam_line = f"{family_ids[i]} {individual_ids[i]} {father_ids[i]} {mother_ids[i]} {sex_codes[i]} {phenotype_values[i]}\n"

    if os.path.exists(fam_file_path):
        with open(fam_file_path, 'w') as fam_file:
            fam_file.write(new_fam_line)
        print(f".fam file updated: {fam_file_path}")
    else:
        raise FileNotFoundError(f".fam file not found: {fam_file_path}")

# Step 3: Merge All PLINK Files
print("Merging PLINK files...")
merge_command = f"/content/plink --bfile {plink_file_prefixes[0]}"
for prefix in plink_file_prefixes[1:]:
    merge_command += f" --bmerge {prefix}"

merged_prefix = f"{output_base}_merged"
merge_command += f" --make-bed --allow-no-sex --out {merged_prefix}"
subprocess.run(merge_command, shell=True, check=True)
print(f"Merged PLINK file created: {merged_prefix}")

# Step 4: Trim Again After Merging if Requested
if trim_with_snplist:
    trimmed_merged_prefix = f"{output_base}_merged_trimmed"
    subprocess.run([
        "/content/plink", "--bfile", merged_prefix, "--extract", snplist_path,
        "--make-bed", "--out", trimmed_merged_prefix
    ], check=True)
    merged_prefix = trimmed_merged_prefix  # Update prefix after trimming

# Step 5: Convert to EIGENSTRAT Format
bedfilepath = f"/content/{merged_prefix}.bed"
bimfilepath = f"/content/{merged_prefix}.bim"
famfilepath = f"/content/{merged_prefix}.fam"
genotypeoutname = f"{output_base}_converted.geno"
snpoutname = f"{output_base}_converted.snp"
indivoutname = f"{output_base}_converted.ind"

convertf_param_content = f"""
genotypename: {bedfilepath}
snpname: {bimfilepath}
indivname: {famfilepath}
genotypeoutname: {genotypeoutname}
snpoutname: {snpoutname}
indivoutname: {indivoutname}
prodercheck: YES
"""
with open("/content/convertf_param.par", "w") as param_file:
    param_file.write(convertf_param_content)
print("convertf parameter file created successfully.")

subprocess.run(["convertf", "-p", "/content/convertf_param.par"], check=True)

# Step 6: Merge with Reference Dataset
mergeit_param_content = f"""
geno1: {geno1_path}
snp1: {snp1_path}
ind1: {ind1_path}
geno2: {genotypeoutname}
snp2: {snpoutname}
ind2: {indivoutname}
genooutfilename: {output_base}_merged.geno
snpoutfilename: {output_base}_merged.snp
indoutfilename: {output_base}_merged.ind
testmismatch: NO
numthreads: {num_threads}
"""
with open("/content/mergeit_param.par", "w") as merge_param_file:
    merge_param_file.write(mergeit_param_content)

print("Running mergeit...")
!mergeit -p /content/mergeit_param.par

# Verify Merging Results
merged_files_exist = all(os.path.exists(f"/content/{output_base}_merged.{ext}") for ext in ["geno", "snp", "ind"])
print("Merging completed successfully." if merged_files_exist else "An error occurred during merging.")

Select dataset (1 for HO, 2 for 1240K): 2
Enter sex for /content/zohan23andme.txt (1 for male, 2 for female, 0 for unknown): 1
Enter name for /content/zohan23andme.txt (to be displayed in .ind file): Zohan.DG
Enter output base prefix: 4Zohan
Do you want to trim using a SNP list? (yes/no): yes
Swap memory created successfully.
Processing /content/zohan23andme.txt...
.fam file updated: /content/4Zohan_1_trimmed.fam
Merging PLINK files...
Merged PLINK file created: 4Zohan_merged
convertf parameter file created successfully.
Running mergeit...
parameter file: /content/mergeit_param.par
geno1: aadr_v62.0_1240K_public.geno
snp1: aadr_v62.0_1240K_public.snp
ind1: aadr_v62.0_1240K_public.ind
geno2: 4Zohan_converted.geno
snp2: 4Zohan_converted.snp
ind2: 4Zohan_converted.ind
genooutfilename: 4Zohan_merged.geno
snpoutfilename: 4Zohan_merged.snp
indoutfilename: 4Zohan_merged.ind
testmismatch: NO
numthreads: 2
## mergeit version: 3130
allele funny: rs2821054 G T X X
allele funny: rs4845877 T C X X


# **[5] Download the merged files as a zip**

Just run the thing and give it the names (same one you gave it to the merged files).

In [ ]:
import os
import zipfile

# Ask the user to specify the base filename for .ind, .geno, and .snp files
base_filename = input("Enter the base filename for the merged files (without extension): ").strip()

if not base_filename:
    raise ValueError("Base filename cannot be empty.")

# Define the file extensions to include
file_extensions = ["geno", "snp", "ind"]

# Define excluded names
excluded_names = ["aadr_v62.0_HO_public", "aadr_v62.0_1240K_public"]

# Create a Zip File with Selected Files
zip_filename = f"{base_filename}_selected.zip"
zip_filepath = f"/content/{zip_filename}"

print(f"Creating ZIP archive: {zip_filename}...")

with zipfile.ZipFile(zip_filepath, 'w') as zipf:
    for ext in file_extensions:
        file_path = f"/content/{base_filename}_merged.{ext}"

        # Skip files that match excluded names
        if os.path.exists(file_path) and not any(excluded in file_path for excluded in excluded_names):
            zipf.write(file_path, os.path.basename(file_path))
        else:
            print(f"Skipping: {file_path} (not found or excluded)")

print(f"ZIP archive created successfully: {zip_filepath}")

# Check if file exists
if os.path.exists(zip_filepath):
    print(f"File found: {zip_filepath}")
    print("File size:", round(os.path.getsize(zip_filepath) / (1024 * 1024), 2), "MB")

    # Trigger download
    print("\nStarting download to your PC...")
    files.download(zip_filepath)
    print("Check your browser for the download prompt!")
else:
    print("Error: File not found at", zip_filepath)
    print("Please make sure:")
    print("1. The file exists in Colab's /content directory")
    print("2. You've run all previous cells that generate this file")
    print("3. The filename format matches '(BASEFILENAME)_merged'")
